In [3]:
# ─── Instalação de dependências ───────────────────────────────────────────────
!pip install -q google-genai gradio schedule tinydb google-api-python-client google-auth-httplib2 google-auth-oauthlib google-adk

import os
import time
import schedule
from datetime import datetime, date
from threading import Thread
import webbrowser

import gradio as gr
from google import genai
from google.genai import types
from tinydb import TinyDB, Query
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

# ─── CONFIGURAÇÕES GERAIS ──────────────────────────────────────────────────────
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# ─── BANCO DE DADOS (TinyDB) ────────────────────────────────────────────────────
db = TinyDB('senior_hub_db.json')
history_table = db.table('history')
reminder_table = db.table('reminders')

# ─── SITES ÚTEIS (DEMONSTRATIVO) ─────────────────────────────────────────────────────
useful_websites = [
    {'name': 'INSS',           'icon': '🏠', 'url': 'https://www.inss.gov.br'},
    {'name': 'SUS',            'icon': '⚕️', 'url': 'https://www.gov.br/saude/pt-br'},
    {'name': 'Receitas',       'icon': '🍳', 'url': 'https://www.receitas.com'},
    {'name': 'Relaxar',        'icon': '🧘‍♀️', 'url': 'https://www.relaxar.com'},
    {'name': 'Gov.br',         'icon': '🌐', 'url': 'https://www.gov.br'},
    {'name': 'Receita Federal','icon': '💰', 'url': 'https://www.gov.br/receitafederal'},
    {'name': 'Embrapa',        'icon': '🌱', 'url': 'https://www.embrapa.br'},
    {'name': 'Globoplay',      'icon': '📺', 'url': 'https://www.globoplay.globo.com'},
    {'name': 'Prefeitura SP',  'icon': '🏙️', 'url': 'https://www.prefeitura.sp.gov.br'},
    {'name': 'DPU',            'icon': '⚖️', 'url': 'https://www.dpu.def.br'}
]

# ─── FUNÇÕES DO CHATBOT ─────────────────────────────────────────────────────────
def load_history():
    rec = history_table.get(Query().id == 'chat')
    return rec['messages'] if rec else []

def save_history(msgs):
    history_table.upsert({'id': 'chat', 'messages': msgs}, Query().id == 'chat')

def respond_to_user(user_message, chat_history):
    history = chat_history or load_history()
    history.append((user_message, None))
    response_agent = Agent(
        name='response_agent', model=MODEL_ID,
        instruction="""Você é um assistente virtual empático e atencioso, especialmente para idosos.
...""",
        tools=[google_search]
    )
    # The call_agent function is not defined in the provided code.
    # It's likely intended to be part of the google.adk library,
    # but it's not a standard function.
    # To fix this, you would need to either:
    # 1. Import or define a call_agent function if it exists in the google.adk library
    #    or elsewhere.
    # 2. Replace call_agent with the correct method for running the agent from google.adk.
    #    Looking at the google-adk library, you would likely use a Runner to run the agent.
    #    Example (assuming you have a Runner instance available, e.g., `runner = Runner()`):
    #    agent_response = runner.run(response_agent, user_message).text
    # For now, I will comment out the line to allow the code to run without the IndentationError,
    # but this function will need to be correctly implemented for the chatbot to work.
    # agent_response = call_agent(response_agent, user_message)
    agent_response = "Função call_agent não implementada." # Placeholder response
    history[-1] = (user_message, agent_response)
    save_history(history)
    return history, ""

def first_interaction():
    greeting = """
Olá! Sou seu Assistente Inteligente para a Melhor Idade... 🤗
"""
    return [[None, greeting]]

# ─── LEMBRETES E AGENDADOR ─────────────────────────────────────────────────────
def schedule_reminder(title: str, time_str: str) -> None:
    reminder_table.insert({'title': title, 'time': time_str})
    schedule.every().day.at(time_str).do(lambda: print(f"⏰ Lembrete: {title}"))

def get_appointments() -> str:
    appts = [r for r in reminder_table.all() if not r['title'].startswith('Medicamento:')]
    if not appts:
        return "Nenhum compromisso agendado."
    lines = [f"{r['time']} - {r['title']}" for r in sorted(appts, key=lambda x: x['time'])]
    return "\n".join(lines)

def get_medications() -> str:
    meds = [r for r in reminder_table.all() if r['title'].startswith('Medicamento:')]
    if not meds:
        return "Nenhum medicamento agendado."
    lines = [f"{r['time']} - {r['title']}" for r in sorted(meds, key=lambda x: x['time'])]
    return "\n".join(lines)

# iniciando scheduler
Thread(target=lambda: (schedule.run_pending(), time.sleep(1)), daemon=True).start()

# ─── INTERFACE GRADIO ──────────────────────────────────────────────────────────
css = """
body { background: #fafafa; font-family: sans-serif; }
#header { text-align: center; margin-bottom: 20px; }
.button { background: #f0f0f0; color: #333; border: 1px solid #ccc; padding: 8px 16px; border-radius: 8px; margin-bottom: 5px; text-align: left; }
.tabitem { font-size: 1.2em; font-weight: bold; }
"""
# The aggregated_news function is not defined in the provided code.
# To fix this, you need to define the aggregated_news function
# that takes a topic (string) and returns a string containing
# the top 3 news related to that topic.
# For now, I will define a placeholder function to allow the code to run.
def aggregated_news(topic: str) -> str:
    """Placeholder function for fetching aggregated news."""
    return f"Função aggregated_news não implementada para o tópico: {topic}"


def launch_app():
    with gr.Blocks(css=css) as demo:
        gr.Markdown("## Assistente para a Melhor Idade 🌟", elem_id='header')
        with gr.Tabs():

            # Conversa
            with gr.TabItem("💬 Conversa"):
                chatbot = gr.Chatbot(value=first_interaction(), height=300)
                user_input = gr.Textbox(placeholder="Digite aqui…")
                user_input.submit(respond_to_user, [user_input, chatbot], [chatbot, user_input])

            # Notícias
            with gr.TabItem("📰 Notícias"):
                topic = gr.Textbox(label="Tópico", placeholder="Ex: saúde, finanças…")
                btn = gr.Button("🔍 Buscar Notícias")
                out = gr.Textbox(label="Top 3 Notícias", lines=6)
                btn.click(aggregated_news, topic, out)

            # Compromissos e Medicamentos
            with gr.TabItem("🗓️ Compromissos e Medicamentos"):
                # Compromissos
                appt_title = gr.Textbox(label="Título do compromisso")
                appt_time  = gr.Textbox(label="Horário (HH:MM)")
                appt_btn   = gr.Button("Adicionar Compromisso")
                appt_list  = gr.Textbox(label="Lista de Compromissos", lines=5, value="Nenhum compromisso agendado.")
                appt_btn.click(lambda t, tm: (schedule_reminder(t, tm), get_appointments())[1],
                               [appt_title, appt_time], appt_list)

                # Medicamentos
                med_name = gr.Textbox(label="Nome do medicamento")
                med_time = gr.Textbox(label="Horário (HH:MM)")
                med_dose = gr.Textbox(label="Dose (ex: 2 comprimidos ou 5ml)")
                med_btn  = gr.Button("Adicionar Medicamento")
                med_list = gr.Textbox(label="Horários de Medicamentos", lines=5, value="Nenhum medicamento agendado.")
                med_btn.click(lambda n, tm, d: (schedule_reminder(f"Medicamento: {n} | Dose: {d}", tm), get_medications())[1],
                              [med_name, med_time, med_dose], med_list)

            # Utilidades
            with gr.TabItem("🔗 Utilidades"):
                for s in useful_websites:
                    gr.Markdown(f"- [{s['icon']} {s['name']}]({s['url']})")

        demo.launch()

if __name__ == '__main__':
    launch_app()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

<ipython-input-3-c399809942cf>:132: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=first_interaction(), height=300)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://05d59a7c749f196a78.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
